In [13]:
from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_upstage import UpstageEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from decouple import config
import pandas as pd
import numpy as np

# 1. 기본 세팅

In [2]:
load_dotenv()

True

In [4]:
pc = Pinecone()

In [5]:
embedding = OpenAIEmbeddings(model="text-embedding-3-large")
index_name = config("PINECONE_INDEX")

In [6]:
database = PineconeVectorStore(
    embedding=embedding, 
    index_name=index_name
)

In [7]:
retriever = database.as_retriever(search_kwargs={"k": 10})
prompt = hub.pull('rlm/rag-prompt')
llm = ChatOpenAI(model="gpt-4.1-nano")

In [8]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = retriever,
    chain_type_kwargs={"prompt": prompt},
)

# 2. 검증을 위한 문제 데이터셋 로드

In [11]:
df = pd.read_csv("법령문제_금융은행보험.csv")
df.sample()

,법령명,문제유형,난이도,문제내용,보기1,보기2,보기3,보기4,정답,해설
1291,중소기업은행법제54조제6항시행에관한규정(대통령령)(제20653호)(20080229),사지선다형,중,금융위원회는 민간인출자의 인수신청의 접수를 누구에게 대행하게 할 수 있는가?,A. 중소기업청장,B. 중소기업은행장,C. 재무부장관,D. 국무총리,2,"제8조에 따르면, 금융위원회는 중소기업은행장에게 대행하게 할 수 있다."


In [ ]:
sample = df.sample()
question = """아래 문제를 보고 답을 고르시오.
만약 보기 중 3번째 보기가 정답이라면 3 이라고 출력하시오.
출력 예시: 3
"""


question += sample['문제내용'].values[0] + '\n'

for i in range(1,5):
    col = '보기' + str(i)
    question += sample[col].values[0] + '\n'

print(question)
print(sample['정답'].values[0])


아래 문제를 보고 답을 고르시오.
만약 보기 중 3번째 보기가 정답이라면 3 이라고 출력하시오.
출력 예시: 3
업무에 관한 약관에 반드시 기재해야 하는 사항이 아닌 것은?
A. 계약해제의 조건
B. 급부금의 지급사유
C. 대출금의 이자율
D. 계약에 관한 권리의 양도

3


In [39]:
message = qa_chain.invoke({"query":question})
print(message['result'])

3


# 3. 실제 검증 실행

In [79]:
success, fail = 0, 0
for i in range(5):
    sample = df.loc[i]
    question = """아래 문제를 보고 답을 고르시오.
만약 보기 중 3번째 보기가 정답이라면 3 이라고 출력하시오.
출력 예시: 3
"""
    question += sample['문제내용'] + '\n'

    for i in range(1,5):
        col = '보기' + str(i)
        question += sample[col] + '\n'

    message = qa_chain.invoke({"query":question})
    if message['result'] == str(sample['정답']):
        success += 1
    else:
        fail += 1

In [82]:
print('정답 갯수 :', success)
print('오답 갯수 :', fail)
print('정확도 :', (success)/(success+fail))


정답 갯수 : 4
오답 갯수 : 1
정확도 : 0.8
